In [2]:
from pathlib import Path
import re
import records
from babel.numbers import parse_decimal, NumberFormatError

schema_re = re.compile(r'\((.+)\)')
num_re = re.compile(r'[-+]?\d*\.\d+|\d+')

db_path = Path("./private")
db = records.Database(f"sqlite:///{db_path / 'samsung_new.db'}")

테이블 명 확인해보기

In [3]:
db.get_table_names()

['company', 'kospi', 'receipts']

계정명 확인해보기

In [6]:
# receipts 를 판다스 테이블로 불러옴
df_receipts = db.query("SELECT * FROM 'receipts'").export("df")
# account_nm 계정명
df_receipts["account_nm"].unique()

array(['유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계', '이익잉여금', '자본총계',
       '매출액', '영업이익', '법인세차감전 순이익', '당기순이익', '자본금'], dtype=object)

각 Column에 해당하는 의미
- rcept_no: 접수번호
- corp_code: 사업 연도
- stock_code: 종목 코드
- reprt_code: 보고서 코드
- account_nm: 계정명 (예: 자본총계)
- fs_div: 개별/연결구분 ('CFS'=연결재무제표, 'OFS'=재무제표)
- fs_nm: 개별/연결명 ('연결재무제표' 또는 '재무제표')
- sj_div: 재무제표구분 ('BS'=재무상태표, 'IS'=손익계산서)
- sj_nm: 재무제표명 ( '재무상태표' 또는 '손익계산서')
- thstrm_nm: 당기명
- thstrm_dt: 당기일자
- thstrm_amount: 당기금액
- thstrm_add_amount: 당기누적금액
- frmtrm_nm: 전기명
- frmtrm_dt: 전기일자
- frmtrm_amount: 전기금액
- frmtrm_add_amount: 전기누적금액
- bfefrmtrm_nm: 전전기명
- bfefrmtrm_dt: 전전일자
- bfefrmtrm_amount: 전전기금액

테이블 스키마 확인

In [9]:
table_id = "receipts"
table_info = db.query('SELECT sql from sqlite_master WHERE tbl_name = :name', name=table_id).all()[0].sql
schema_str = schema_re.findall(table_info.replace("\n", ""))[0]
schema = {}
for tup in schema_str.split(', '):
    c, t = tup.split()
    schema[c.strip('"')] = t
schema

{'index': 'INTEGER',
 'rcept_no': 'TEXT',
 'reprt_code': 'TEXT',
 'bsns_year': 'INTEGER',
 'corp_code': 'TEXT',
 'stock_code': 'TEXT',
 'fs_div': 'TEXT',
 'fs_nm': 'TEXT',
 'sj_div': 'TEXT',
 'sj_nm': 'TEXT',
 'account_nm': 'TEXT',
 'thstrm_nm': 'TEXT',
 'thstrm_dt': 'TEXT',
 'thstrm_amount': 'INTEGER',
 'frmtrm_nm': 'TEXT',
 'frmtrm_dt': 'TEXT',
 'frmtrm_amount': 'INTEGER',
 'bfefrmtrm_nm': 'TEXT',
 'bfefrmtrm_dt': 'TEXT',
 'bfefrmtrm_amount': 'INTEGER'}

In [7]:
# receipts 테이블의 칼람명: SELECT 할 때 칼럼명을 불러야함
df_receipts.columns

Index(['index', 'rcept_no', 'reprt_code', 'bsns_year', 'corp_code',
       'stock_code', 'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm',
       'thstrm_nm', 'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt',
       'frmtrm_amount', 'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount'],
      dtype='object')

쿼리가 맞게 써졌는지 확인하는 방법:

- 한글은 '' 로 감싸줘야함
- 삼성전자의 2020년도의 유동자산은 어떻게 돼? 
    - SELECT는 당기금액인 thstrm_amount 선택해야하고
    - FROM 은 테이블 명인 receipts
    - WHERE 문은 조건인 2020년도면 회계연도인 bsns_year = 2020 와 조회하고 싶은 계정명 account_nm = '유동자산' 

In [8]:
db.query("SELECT thstrm_amount FROM receipts WHERE account_nm = '유동자산' AND bsns_year = 2020").all()[0].as_dict()

{'thstrm_amount': 198215579000000}

판다스 테이블로 직접 조회해서 확인해보는 방법도 있음

In [15]:
df_receipts.loc[(df_receipts["bsns_year"] == 2020) & (df_receipts["account_nm"] == "유동자산"), "thstrm_amount"]

60    198215579000000
Name: thstrm_amount, dtype: int64

## 만들어야할 형태

tsv 파일에 다음과 같은 string을 print 하면 됨

In [17]:
s = [
    "삼성전자의 2020년도의 유동자산은 어떻게 돼?",  # Question
    "SELECT thstrm_amount FROM receipts WHERE account_nm = '유동자산' AND bsns_year = 2020",  # SQL
    "198215579000000"  # Answer
] 
print("\t".join(s))

삼성전자의 2020년도의 유동자산은 어떻게 돼?	SELECT thstrm_amount FROM receipts WHERE account_nm = '유동자산' AND bsns_year = 2020	198215579000000


---

## 데이터 예시

우선 terminology, `[시간]`, `[항목]`의 토큰을 정의하고, 그 다음에 unique한 자연어 질의 먼저 생성하고 SQL을 만들어야 할듯

- 자연어 질의: 제 51 기에 삼성전자의 이익잉여금은 어떻게 돼?
- SQL: 
    ```SQL
    /* Possible answer 1 */
    SELECT frmtrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND bsns_year = 2020
    /* Possible answer 2 */
    SELECT thstrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND bsns_year = 2019
    /* Possible answer 3 */
    SELECT thstrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND thstrm_nm = '제 51 기'
    /* Possible answer 4 */
    SELECT frmtrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND frmtrm_nm = '제 51 기'
    ```

<details>
<summary>Jsonl 형태:</summary>

```json
{
   "phase":1,
   "question":"제 51 기에 삼성전자의 유동자산은 어떻게 돼?",
   "sql":{
      "conds":[
         [10, 0, "이익잉여금"], [3, 0, 2020]
      ],
      "sel":16,
      "agg":0
   },
   "table_id":"receipts"
}
{
   "phase":1,
   "question":"제 51 기에 삼성전자의 유동자산은 어떻게 돼?",
   "sql":{
      "conds":[
         [10, 0, "이익잉여금"], [3, 0, 2019]
      ],
      "sel":13,
      "agg":0
   },
   "table_id":"receipts"
}
...
```
</details>

- `phase`: the phase in which the dataset was collected. We collected WikiSQL in two phases.
- `question`: the natural language question written by the worker.
- `table_id`: the ID of the table to which this question is addressed.
- `sql`: the SQL query corresponding to the question. This has the following subfields:
  - `sel`: the numerical index of the column that is being selected. You can find the actual column from the table.
  - `agg`: the numerical index of the aggregation operator that is being used. You can find the actual operator from `Query.agg_ops` in `lib/query.py`.
  - `conds`: a list of triplets `(column_index, operator_index, condition)` where:
    - `column_index`: the numerical index of the condition column that is being used. You can find the actual column from the table.
    - `operator_index`: the numerical index of the condition operator that is being used. You can find the actual operator from `Query.cond_ops` in `lib/query.py`.
    - `condition`: the comparison value for the condition, in either `string` or `float` type.

질문: 

1. 계정항목?
2. 추가 간단한 질문들?

In [96]:
sqls = [
    "SELECT frmtrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND bsns_year = 2020",
    "SELECT thstrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND bsns_year = 2019",
    "SELECT thstrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND thstrm_nm = '제 51 기'",
    "SELECT frmtrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND frmtrm_nm = '제 51 기'"
]
for sql in sqls:
    res = db.query(sql)
    print(res.as_dict()[0])

{'frmtrm_amount': 254582894000000}
{'thstrm_amount': 254582894000000}
{'thstrm_amount': 254582894000000}
{'frmtrm_amount': 254582894000000}


In [167]:
import json
from typing import Union
from moz_sql_parser import parse as sql_parser
schema_re = re.compile(r'\((.+)\)')

class Generator:
    
    agg_ops = ["", "MAX", "MIN", "COUNT", "SUM", "AVG"]
    cond_ops = ["=", ">", "<", "OP", ">=", "<="]
    cond_ops_dict = {"eq": "=", "lt": "<",  "lte": "<=", "gt": ">", "gte": ">=", "neq": "<>"}

    syms = ["SELECT", "WHERE", "AND", "COL", "TABLE", "CAPTION", "PAGE", "SECTION", "OP", "COND", "QUESTION", "AGG", "AGGOPS", "CONDOPS"]
    
    def __init__(self, db_path: Union[Path, str]) -> None:
        self.db = records.Database(f"sqlite:///{db_path}")
        self._reset()
        
    def _reset(self) -> None:
        self.table_id = None
        self.schema = None
        self.col2idx = None
        
    def get_schema_info(self, table_id: str) -> None:
        table_info = self.db.query('SELECT sql from sqlite_master WHERE tbl_name = :name', name=table_id).all()[0].sql
        schema_str = schema_re.findall(table_info.replace("\n", ""))[0]
        schema = {}
        for tup in schema_str.split(', '):
            c, t = tup.split()
            schema[c.strip('"')] = t
        col2idx = {c: i for i, c in enumerate(schema.keys())}
        
        self.table_id = table_id
        self.schema = schema
        self.col2idx = col2idx
    
    def to_jsonl(self, sql: str, question: str) -> dict:
        r"""
        # Only 1 agg and select
        example:
        - sql: "SELECT frmtrm_amount FROM receipts WHERE account_nm = '이익잉여금' AND bsns_year = 2020",
        - question: "제 51 기에 삼성전자의 유동자산은 어떻게 돼?"
        
        return:
        {
           "phase":1,
           "question":"제 51 기에 삼성전자의 유동자산은 어떻게 돼?",
           "sql":{
              "conds":[
                 [10, 0, "이익잉여금"], [3, 0, 2020]
              ],
              "sel":16,
              "agg":0
           },
           "table_id":"receipts"
        }
        
        """
        
        parsed = sql_parser(sql)
        table_id = parsed["from"]
        jsonl = {"phase": 1, "question": question, "table_id": table_id, "sql": {}}
        
        if (self.table_id is None) or (self.table_id != table_id):
            self.get_schema_info(table_id)
#         else:
#             raise AttributeError("No schema information, please make sure to call `self.get_schema_info`")
        
        select_parsed = parsed["select"]["value"]
        if isinstance(select_parsed, dict):
            # Only 1 agg and select
            agg_name = list(select_parsed)[0]
            agg = self.agg_ops.index(agg_name.upper())
            select_name = select_parsed[agg]
        elif isinstance(select_parsed, str):
            agg = 0
            select_name = select_parsed
        else:
            raise TypeError(f"Parsed in select clause should be `str` or `dict` type, Current is {select_parsed}")
        select = self.col2idx.get(select_name)
        
        conds_parsed = parsed["where"]
        conds = []
        for operator, conditions in conds_parsed.items():
            cond = {operator.upper(): []}
            for condition in conditions:
                key, values = tuple(condition.items())[0]

                if self.cond_ops_dict.get(key) is None:
                    raise KeyError(f"No operator: {key}")
                else:
                    op = self.cond_ops_dict.get(key)
                    op_idx = self.cond_ops.index(op)
                    
                if self.col2idx.get(values[0]) is None:
                    raise KeyError(f"No column name: {values[0]}")
                else:
                    col_idx = self.col2idx.get(values[0])
                    
                
                if isinstance(values[1], dict):
                    # make sure all string values insert '' when parse to sql again
                    cond_value = values[1]["literal"]
                else:
                    cond_value = values[1]
                cond[operator.upper()].append([col_idx, op_idx, cond_value])
            conds.append(cond)
        
        jsonl["sql"]["sel"] = select
        jsonl["sql"]["agg"] = agg
        jsonl["sql"]["conds"] = conds
        return jsonl
        

In [168]:
sql_gen = Generator(db_path=db_path / "samsung_new.db")

In [169]:
sql_gen.schema

In [170]:
jsons = []
for sql in sqls:
    jsons.append(sql_gen.to_jsonl(sql, "제 51 기에 삼성전자의 이익잉여금은 어떻게 돼?"))

In [171]:
jsons

[{'phase': 1,
  'question': '제 51 기에 삼성전자의 유동자산은 어떻게 돼?',
  'table_id': 'receipts',
  'sql': {'sel': 16,
   'agg': 0,
   'conds': [{'AND': [[10, 0, '이익잉여금'], [3, 0, 2020]]}]}},
 {'phase': 1,
  'question': '제 51 기에 삼성전자의 유동자산은 어떻게 돼?',
  'table_id': 'receipts',
  'sql': {'sel': 13,
   'agg': 0,
   'conds': [{'AND': [[10, 0, '이익잉여금'], [3, 0, 2019]]}]}},
 {'phase': 1,
  'question': '제 51 기에 삼성전자의 유동자산은 어떻게 돼?',
  'table_id': 'receipts',
  'sql': {'sel': 13,
   'agg': 0,
   'conds': [{'AND': [[10, 0, '이익잉여금'], [11, 0, '제 51 기']]}]}},
 {'phase': 1,
  'question': '제 51 기에 삼성전자의 유동자산은 어떻게 돼?',
  'table_id': 'receipts',
  'sql': {'sel': 16,
   'agg': 0,
   'conds': [{'AND': [[10, 0, '이익잉여금'], [14, 0, '제 51 기']]}]}}]